In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/model-train/covid_toy.csv


In [20]:
import numpy as np
import pandas as pd

In [21]:
df = pd.read_csv("/kaggle/input/models/covid_toy.csv")
df.head() 

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [22]:
 df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [23]:
from sklearn.impute import SimpleImputer  

In [24]:
si = SimpleImputer(strategy = "mean")

In [25]:
df['fever'] = si.fit_transform(df[['fever']])

In [26]:
 df.isnull().sum()

age          0
gender       0
fever        0
cough        0
city         0
has_covid    0
dtype: int64

In [27]:
df['has_covid'].value_counts()

has_covid
No     55
Yes    45
Name: count, dtype: int64

In [28]:
print(df.columns)

Index(['age', 'gender', 'fever', 'cough', 'city', 'has_covid'], dtype='object')


In [29]:
df['gender'] = df['gender'].map({"Female":1 ,"Male":0})
df['cough'] = df['cough'].map({"Mild":0 , "Strong":1})
df['city'] = df['city'].map({"Kolkata":0 , "Bangalore":1 , "Delhi":2 , "Mumbai":3})
df['has_covid'] = df['has_covid'].map({"No":0 , "Yes":1})

In [30]:
x = df.drop(columns = ['has_covid'] , axis = 1)
y = df['has_covid']

In [31]:
df.head(3)

,age,gender,fever,cough,city,has_covid
0,60,0,103.0,0,0,0
1,27,0,100.0,0,2,1
2,42,0,101.0,0,2,0


In [32]:
from sklearn.model_selection import train_test_split

In [33]:
x_train, x_test , y_train , y_test = train_test_split(x,y,
                                                      test_size=0.2,
                                                      random_state=42)

In [62]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)

param_dist = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
}



random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist,
                                   n_iter=20, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)
random_search.fit(x_train, y_train)
best_model = random_search.best_estimator_


In [63]:
rf = RandomForestClassifier()

In [64]:
rf.fit(x_train , y_train)

RandomForestClassifier()

In [65]:
y_pred = rf.predict(x_test)

In [66]:
from sklearn.metrics import accuracy_score

In [67]:
accuracy_score(y_test , y_pred)

0.5